In [32]:
from transformers import AutoTokenizer, LlamaConfig, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorForLanguageModeling
from peft import get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer,GenerationConfig
from datasets import Dataset , load_dataset
import torch
import accelerate
from transformers.generation.utils import GreedySearchDecoderOnlyOutput
import textwrap
#import bitsandbytes
import pickle

In [2]:
#torch.cuda.empty_cache()


In [3]:
path="/home/vicuna-weights-7B"
tokenizer = LlamaTokenizer.from_pretrained(path)
#tokenizer.pad_token = tokenizer.eos_token
#okenizer.pad_token_id = tokenizer.eos_token_id
#model = LlamaForCausalLM.from_pretrained(path,load_in_8bit=True,device_map='auto',)


In [4]:
# "/home/vicuna-weights-7B" or /home/final_llama
torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = LlamaForCausalLM.from_pretrained(path)
torch.set_default_tensor_type(torch.cuda.FloatTensor)

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.13s/it]


In [5]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [6]:
model = model.eval()
model = torch.compile(model)

In [ ]:
#tokenizer.pad_token = tokenizer.eos_token

In [45]:
stop=["#",'##','###','</s>', ":",'?'] 
ques='what is the capital of india ?'

In [51]:
#output = compute_until_stop(model, tokenizer, params, device='cuda',)

In [7]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.
 
### Instruction:
[INSTRUCTION]
 
### Response:
"""

In [16]:
def create_prompt(instruction):
    return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)
 
#print(create_prompt("What is the meaning of life?"))

In [37]:
def generate_response(prompt, model):
    encoding = tokenizer(prompt, return_tensors="pt")
    input_ids = encoding["input_ids"].to('cuda')
 
    generation_config = GenerationConfig(
        temperature=0.5,
        top_p=0.75,
        repetition_penalty=1.1,
        beam_size=1,
        max_length=20,
        min_length=1,
        
    )
    with torch.inference_mode():
        return model.generate(
            input_ids=input_ids,
            generation_config=generation_config,
            return_dict_in_generate=True,
            output_scores=True,
            max_new_tokens=20,
        )

In [38]:
def format_response(response):
    decoded_output = tokenizer.decode(response.sequences[0])
    response = decoded_output.split("### Response:")[1].strip()
    return "\n".join(textwrap.wrap(response))

In [39]:
def ask_vicuna(prompt, model=model):
    prompt = create_prompt(prompt)
    response = generate_response(prompt, model)
    b=format_response(response)
    a=b.split('###')
    return a[0]

In [23]:
question="Explain the concept of metabolic engineering and discuss a complex case where it has been successfully applied."

In [30]:

prompting=f"""
{question}
"""
print(ask_vicuna(prompting))

In [40]:
context='The Norman dynasty had a major political, cultural and military impact on medieval Europe and even the Near East. The Normans were famed for their martial spirit and eventually for their Christian piety, becoming exponents of the Catholic orthodoxy into which they assimilated. They adopted the Gallo-Romance language of the Frankish land they settled, their dialect becoming known as Norman, Normaund or Norman French, an important literary language. The Duchy of Normandy, which they formed by treaty with the French crown, was a great fief of medieval France, and under Richard I of Normandy was forged into a cohesive and formidable principality in feudal tenure. The Normans are noted both for their culture, such as their unique Romanesque architecture and musical traditions, and for their significant military accomplishments and innovations. Norman adventurers founded the Kingdom of Sicily under Roger II after conquering southern Italy on the Saracens and Byzantines, and an expedition on behalf of their duke, William the Conqueror, led to the Norman conquest of England at the Battle of Hastings in 1066. Norman cultural and military influence spread from these new European centres to the Crusader states of the Near East, where their prince Bohemond I founded the Principality of Antioch in the Levant, to Scotland and Wales in Great Britain, to Ireland, and to the coasts of north Africa and the Canary Islands.'
question='What type of major impact did the Norman dynasty have on modern Europe?'

In [41]:
template = f"""
Your task is to read the following passage and answer the subsequent question based on the information provided in the passage.
rules for answering questions:
1. The answer must be a few words in length , single word to few words -max 20 words.
2. follow the ordering of the passage while generating an answer.
3. Don't add any words which are not in the passage.
4. If a question is not answerable from the passage, write "not answerable" as the answer.

Passage: {context}

Question: {question}


Answer: 
"""


In [42]:
print(ask_vicuna(template))

Military. 
